In [5]:
import ax
import os
import time 
from typing import Any, Mapping
import subprocess


import numpy as np 
from ax.api.client import Client
from ax.api.configs import RangeParameterConfig
from ax.api.protocols.metric import IMetric
from ax.api.protocols.runner import IRunner, TrialStatus
from ax.api.types import TParameterization 
import json

In [2]:
from ax import Client, RangeParameterConfig

In [3]:
client= Client()

<font SIZE= 6> Parameter Configuration

In [4]:
parameters= [
    RangeParameterConfig(
        name="x1", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x2", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x3", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x4", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x5", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x6", parameter_type="float", bounds=(0, 1)
    ),
]

Client.configure_experiment(parameters=parameters)

TypeError: Client.configure_experiment() missing 1 required positional argument: 'self'

<font size= 6> Main Function

In [ ]:
def blackbox_function(x1,x2, x3, x4, x5, x6): 
    #This is where the experiment task happens
    #delegate the task of printing to the external system (computer and the AJP)
    #return the set of values (for now, line width) to the system

<FONT SIZE = 6> Mimicking interaction (Deploying and Fetching) with an external system (IRunner)

In [ ]:
INPUT_DIR = "C://AJP/input_parameters"
OUTPUT_DIR = "C://AJP/output_parameters"

class Runner(IRunner):

    #run_trial deploys a trial to the external system with the given parameters. 
    # Maybe my deploying would be to give a save a JSON file in a particular location or pass it on to an agent  in the workflow


    POWER_AUTOMATE_FLOW_ID = "AUTOMATE_FLOW_ID"
    def run_trial(
        self, trial_index: int, parameterization: TParameterization ) -> dict[str, Any]:
        os.makedirs(INPUT_DIR, exist_ok= True)
        os.makedirs(OUTPUT_DIR, exist_ok = True)

        input_file_path = os.path.join(OUTPUT_DIR, f"params_{trial_index}.json")
        with open(input_file_path, "w") as f: 
            json.dump(parameterization, f, indent=4)

        output_file_path = os.path.join(OUTPUT_DIR, f"results_{trial_index}.json")
        command = [
            "C:\\Program Files (x86)\\Power Automate Desktop\\PAD.Console.Host.exe", # Default path, verify on your system
            "run",
            "--flowId", POWER_AUTOMATE_FLOW_ID,
            "--input", f"InputFilePath={input_file_path}"
        ]

        procerss = subprocess.Popen(
            
        )


        result = blackbox_function(x1, x2, x3, x4, x5, x6)

        
        with open(file_name, "w") as f: 
            f.write(f"{result}")

        return {"file_name": file_name}


    #poll_trial queries the external system to see if the trial has completed, failed, or if it's still runing. There is room to explore how this can be
    # done or if this is needed at all. 
    
    def poll_trial(self, trial_Index :int, trial_metadata: Mapping[str, Any]) -> TrialStatus: 
        file_name = trial_metadata["file_name"]
        time_elapsed = time.time() - int(file_name[:4])

        if time_elapsed < 5: 
            return TrialStatus.Running

        return TrialStatus.COMPLETED

NameError: name 'TParameterization0' is not defined

<font size = 6> 

In [ ]:
class MockMetric(IMetric): 
    def fetch(
        self, 
        trial_index: int, 
        trial_metadata: Mapping[str, Any], ) -> tuple[int, float | tupl[float, float] ]:

        file_name = trial_metadata["file_name"]

        with open (file_name,  'r') as file: 
            value = float(file.readline())
            return (0, value)

<font size =6> Optimization Loop

In [ ]:
for _ in range(10): # Run 10 rounds of trials
    
    
    # We will request three trials at a time in this example
    trials = client.get_next_trials(max_trials= 3) 

    
    for trial_index, parameters in trials.items():
        x1 = parameters["x1"]
        x2 = parameters["x2"]
        x3 = parameters["x3"]
        x4 = parameters["x4"]
        x5 = parameters["x5"]
        x6 = parameters["x6"]


        #This is where the main experimentation part happens: Where I delegate the task to the outer system
        result = hartmann6(x1, x2, x3, x4, x5, x6)

        # Set raw_data as a dictionary with metric names as keys and results as values
        raw_data = {metric_name: result}


        
        # Complete the trial with the result 
        
        #[client.complete_trial(parameters, result]
        #This is kept Ax in the database to generate data for the nextstep        
        client.complete_trial(trial_index=trial_index, raw_data=raw_data)

        print(f"Completed trial {trial_index} with {raw_data=}")

<font size = 6> Fetching the best possible parameters after the given number of runs 

In [ ]:
class MockMetric(IMetric): 
    def fetch(
        self, 
        trial_index: int, 
        trial_metadata: Mapping[str, Any], ) -> tuple[int, float | tupl[float, float] ]:

        file_name = trial_metadata["file_name"]

        with open (file_name,  'r') as file: 
            value = float(file.readline())
            return (0, value)

In [ ]:
client = Client()

parameters= [
    RangeParametersConfig(name = f"x{i + 1}", parameter_type = "float", bounds =(0,1))
    for i in range(6)
]

client.configure_experiment(
    parameters = parameters, 
    name = "hartmann6_experiment", 
    description = "Optimization of the Hartmann6 function", 
    owner = "developer", 
)

client.configure_optimization(objective = "-hartmann6")